机器学习模型将向量作为输入，当处理文本时，将文本转换为数字或者向量，教程会介绍三种方法
- one-hot encodings
- Encode each word with a unique number  (不推荐)
- word embedding

1 one-hot encoding
![onehot](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/images/one-hot.png?raw=1)

当前词为1，其他词为0，稀疏矩阵

2 Encode each word with a unique number 
将词汇转换为：[5,1,4,3,5,2]

3 word embedding
![1](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/images/embedding2.png?raw=1)
四维向量表

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_datasets as tfds
tfds.disable_progress_bar()

D:\Anaconda\envs\tensorflow2.0\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Anaconda\envs\tensorflow2.0\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Anaconda\envs\tensorflow2.0\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Anaconda\envs\tensorflow2.0\lib\site-packages\tensorflow\python\framework\dtypes.py:5

使用 Embedding 层，(input_dim, output_dim),矩阵内显示词与词之间的相似度

In [8]:
# use embedding layer
embedding_layer = layers.Embedding(1000, 5)

In [9]:
result = embedding_layer(tf.constant([1,2,3]))
result.numpy()

array([[ 0.02148049,  0.04827905,  0.03635435, -0.02905328, -0.02243407],
       [ 0.02733265, -0.0094207 ,  0.03874959,  0.04710032,  0.00381619],
       [ 0.03667216,  0.02470085, -0.01392237, -0.03040322,  0.04929138]],
      dtype=float32)

 feed into the embedding layer above batches with shapes (32, 10) (batch of 32 sequences of length 10) or (64, 15) (batch of 64 sequences of length 15).

In [10]:
result = embedding_layer(tf.constant([[0,1,2],[3,4,5]]))
result.shape

TensorShape([2, 3, 5])

In [11]:
result

<tf.Tensor: id=66, shape=(2, 3, 5), dtype=float32, numpy=
array([[[ 0.00174373,  0.03546363, -0.01467752, -0.03533155,
          0.0450004 ],
        [ 0.02148049,  0.04827905,  0.03635435, -0.02905328,
         -0.02243407],
        [ 0.02733265, -0.0094207 ,  0.03874959,  0.04710032,
          0.00381619]],

       [[ 0.03667216,  0.02470085, -0.01392237, -0.03040322,
          0.04929138],
        [ 0.01589671, -0.04374571, -0.04060044,  0.01502068,
          0.04692043],
        [ 0.02178058,  0.0495902 , -0.01148963, -0.03226985,
         -0.030915  ]]], dtype=float32)>

When given a batch of sequences as input, an embedding layer returns a 3D floating point tensor, of shape (samples, sequence_length, embedding_dimensionality)

实际操作，转换文本

In [12]:
(train_data, test_data), info = tfds.load(
    'imdb_reviews/subwords8k', 
    split = (tfds.Split.TRAIN, tfds.Split.TEST), 
    with_info=True, as_supervised=True)

Shuffling and writing examples to C:\Users\liu\tensorflow_datasets\imdb_reviews\subwords8k\1.0.0.incomplete82V5GB\imdb_reviews-train.tfrecord
Shuffling and writing examples to C:\Users\liu\tensorflow_datasets\imdb_reviews\subwords8k\1.0.0.incomplete82V5GB\imdb_reviews-test.tfrecord
Shuffling and writing examples to C:\Users\liu\tensorflow_datasets\imdb_reviews\subwords8k\1.0.0.incomplete82V5GB\imdb_reviews-unsupervised.tfrecord
Dataset imdb_reviews downloaded and prepared to C:\Users\liu\tensorflow_datasets\imdb_reviews\subwords8k\1.0.0. Subsequent calls will reuse this data.


In [13]:
encoder = info.features['text'].encoder
encoder.subwords[:20]

['the_',
 ', ',
 '. ',
 'a_',
 'and_',
 'of_',
 'to_',
 's_',
 'is_',
 'br',
 'in_',
 'I_',
 'that_',
 'this_',
 'it_',
 ' /><',
 ' />',
 'was_',
 'The_',
 'as_']

In [23]:
train_batches = train_data.padded_batch(10)
test_batches = test_data.padded_batch(10)

TypeError: padded_batch() missing 1 required positional argument: 'padded_shapes'

In [20]:
train_batch, train_labels = next(iter(train_batches))
train_batch.numpy()

NameError: name 'train_batches' is not defined

In [18]:
embedding_dim=16

model = keras.Sequential([
  layers.Embedding(encoder.vocab_size, embedding_dim),
  layers.GlobalAveragePooling1D(),
  layers.Dense(16, activation='relu'),
  layers.Dense(1)
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 16)          130960    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 131,249
Trainable params: 131,249
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(
    train_batches,
    epochs=10,
    validation_data=test_batches, validation_steps=20)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


NameError: name 'train_batches' is not defined